In [5]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
import pandas as pd
import os, os.path


raid_train_dir = '/mnt/raid0/Projects/Kaggle/GoogleLandmarkRecognition/train/'
raid_test_dir = '/mnt/raid0/Projects/Kaggle/GoogleLandmarkRetrieval/test/'
train_csv = '~/Documents/Kaggle/GoogleLandmarkRecognition/train.csv'
test_csv = '~/Documents/Kaggle/GoogleLandmarkRecognition/test.csv'

# use this function to load the train and test data
def load_dataset(path):
    data = load_files(path)
    image_files = np.array(data['filenames'])
    train_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return image_files, train_targets

# point this to the train.csv file
def load_variable_names(path):
    # use dtype=None because we have strings and ints
    data = pd.read_csv(path, quotechar='"')
    return data

# use this to count the images available to learn from
def get_total_files(path):
    return len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))])

#train_files, train_targets = load_dataset('/mnt/raid0/Projects/Kaggle/GoogleLandmarkRecognition')
file_info_train = load_variable_names(train_csv)
file_info_test = load_variable_names(test_csv)
total_files_train = get_total_files(raid_train_dir)
total_files_test = get_total_files(raid_test_dir)

print('There are %d files in the train/ dir' % total_files_train)
print('There are %d files in the train.csv file' % len(file_info_train['id']))
print('There are %d unique landmarks in the train.csv file' % len(file_info_train['landmark_id'].unique()))
print('\nThere are %d files in the test/ dir' % total_files_test)
print('There are %d files in the test.csv file' % len(file_info_test['id']))

print('\nMissing %d training files!' % (len(file_info_train['id']) - total_files_train))
print('Missing %d testing files!' % (len(file_info_test['id']) - total_files_test))
    

There are 1224338 files in the train/ dir
There are 1225029 files in the train.csv file
There are 14951 unique landmarks in the train.csv file

There are 117697 files in the test/ dir
There are 117703 files in the test.csv file

Missing 691 training files!
Missing 6 testing files!


In [ ]:
# Need to test and configure this code!!
from keras import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        raid_train_dir,
        target_size=(256, 256),
        color_mode='grayscale',
        batch_size=1000,
        class_mode='categorical')